In [1]:
import cv2
import numpy as np

In [15]:
# load video path
video_path = "sample_videos\Autonomous driving lane detection sample video 1.mp4"
cap = cv2.VideoCapture(video_path)


# change image frame to gray scale
def gray_scale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# apply gaussian blur to the image
def gaussian_blur(frame):
    return cv2.GaussianBlur(frame, (5, 5), 0)

# apply canny edge detection to the image
def canny_edge(frame):
    return cv2.Canny(frame, 50, 150)

# Apply a trapezoidal mask to the frame
def region_of_interest(frame):
    height, width = frame.shape[:2]
    mask = np.zeros_like(frame)

    # Define a trapezoid region of interest
    polygon = np.array([
        [
            (int(width * 0.2), height), # bottom left
            (int(width * 0.8), height), # bottom right
            (int(width * 0.65), int(height * 0.57)), # top right
            (int(width * 0.45), int(height * 0.57)) # top left
        ]
    ], dtype=np.int32)

    # Fill the mask with the polygon
    cv2.fillPoly(mask, polygon, 255)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame

# Change image frame to gray scale
def gray_scale(frame):
    return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to the image
def gaussian_blur(frame):
    return cv2.GaussianBlur(frame, (5, 5), 0)

# Apply Canny edge detection to the image
def canny_edge(frame):
    return cv2.Canny(frame, 20, 40)

# Apply a trapezoidal mask to the frame
def region_of_interest(frame):
    height, width = frame.shape[:2]
    mask = np.zeros_like(frame)

    # Define a trapezoid region of interest
    polygon = np.array([
        [
            (int(width * 0.2), height), # bottom left
            (int(width * 0.8), height), # bottom right
            (int(width * 0.65), int(height * 0.6)), # top right
            (int(width * 0.45), int(height * 0.6)) # top left
        ]
    ], dtype=np.int32)

    # Fill the mask with the polygon
    cv2.fillPoly(mask, polygon, 255)

    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(frame, mask)
    return masked_frame
# Calculate curvature radius
def calculate_curvature(poly, y_vals):
    # Derivatives of the polynomial
    first_derivative = 2 * poly[0] * y_vals + poly[1]
    second_derivative = 2 * poly[0]
    curvature_radius = (1 + first_derivative**2)**1.5 / np.abs(second_derivative + 1e-6)
    return curvature_radius

    
# Fit a polynomial to lane lines and draw them
def fit_polynomial(masked_edge, original_frame):
    # Find lane pixel indices
    lane_pixels = np.column_stack(np.where(masked_edge > 0))
    if len(lane_pixels) == 0:
        return np.zeros_like(original_frame)

    # Separate left and right lane pixels
    height, width = masked_edge.shape
    midpoint = width // 2 + 100
    left_lane_pixels = lane_pixels[lane_pixels[:, 1] < midpoint]
    right_lane_pixels = lane_pixels[lane_pixels[:, 1] >= midpoint]

    def fit_and_draw_lane(pixels, color):
        if len(pixels) == 0:
            return
        y = pixels[:, 0]
        x = pixels[:, 1]
        poly = np.polyfit(y, x, 3)  # Fit a third-degree polynomial

        # Generate points along the fitted polynomial
        # Calculate curvature and adjust drawing range
        y_vals_full = np.linspace(int(height / 1.5), height - 1, num=int(height / 3), dtype=int)
        curvature = calculate_curvature(poly, y_vals_full)
        avg_curvature = np.mean(curvature)

        # Adjust range based on curvature (shorter for tight curves)
        if avg_curvature < 2000:  # Example threshold for tight curves
            y_vals = np.linspace(int(height * 0.75), height - 1, num=int(height * 0.25), dtype=int)
        else:
            y_vals = np.linspace(int(height / 1.5), height - 1, num=int(height / 3), dtype=int)

        x_vals = (poly[0] * y_vals**3 + poly[1] * y_vals**2 + poly[2] * y_vals + poly[3]).astype(int)
        # Draw the lane line
        valid_indices = (x_vals >= 0) & (x_vals < width)
        x_vals = x_vals[valid_indices]
        y_vals = y_vals[valid_indices]
        for i in range(len(x_vals) - 1):
            cv2.line(original_frame, (x_vals[i], y_vals[i]), (x_vals[i + 1], y_vals[i + 1]), color, 10)

    # Fit and draw left and right lanes
    fit_and_draw_lane(left_lane_pixels, (255, 0, 0))  # Left lane in blue
    fit_and_draw_lane(right_lane_pixels, (0, 255, 0))  # Right lane in green

    return original_frame

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # convert to gray scale
    gray = gray_scale(frame)

    # apply gaussian blur
    blur = gaussian_blur(gray)

    # apply canny edge detection
    edge = canny_edge(blur)

    # Apply the region of interest mask
    masked_edge = region_of_interest(edge)

    # Apply Hough transform to detect lane lines
    lines = fit_polynomial(masked_edge, frame)

    # Combine with the original frame
    output = cv2.addWeighted(frame, 0.8, lines, 1, 1)

    # Show the video
    cv2.imshow("Lane Lines", output)


    # video 60 fps, wait according to fps
    cv2.waitKey(int(1000/60))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break



